In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

/Users/huanghailong/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


初始化参数

In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev = 0.1)
    return tf.Variable(initial)

In [4]:
def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)

定义卷积,filter 在W中背决定，W的四个维度分别为[f,f,pre_n_c,filters]。

第一/二个filter的大小，第三个前一层channel数量，第四个该层用多少个filter。

In [5]:
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME')

max_pool ksize 第一个参数 是batch_size, 第二个和第三个是filter大小，2x2。第三个是channel

In [6]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

创造placeholder 接收输入

In [7]:
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
x_image = tf.reshape(x, [-1,28,28,1])# -1 是batch_size，28*28图像大小。1一个channel

In [8]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [9]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [10]:
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [11]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [12]:
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_hat = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [13]:
cross_entropy = tf.reduce_mean(-tf.reduce_mean(y * tf.log(y_hat),reduction_indices=[1]))

In [14]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [15]:
correct_prediction = tf.equal(tf.arg_max(y_hat, 1), tf.arg_max(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:
Use `argmax` instead


In [16]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer().run()

In [17]:
for i in range(2000):
    batch = mnist.train.next_batch(100)
    
    if i % 100 == 0 :
        train_accuracy = accuracy.eval(feed_dict={x:batch[0],y:batch[1],keep_prob:1.0})
        print("step %d, train accuracy %g"%(i,train_accuracy))
    train_step.run(feed_dict={x:batch[0],y:batch[1],keep_prob:0.5})

step 0, train accuracy 0.11
step 100, train accuracy 0.91
step 200, train accuracy 0.91
step 300, train accuracy 0.92
step 400, train accuracy 0.96
step 500, train accuracy 0.93
step 600, train accuracy 0.99
step 700, train accuracy 0.98
step 800, train accuracy 0.94
step 900, train accuracy 0.97
step 1000, train accuracy 0.99
step 1100, train accuracy 0.95
step 1200, train accuracy 0.96
step 1300, train accuracy 0.97
step 1400, train accuracy 0.95
step 1500, train accuracy 0.99
step 1600, train accuracy 0.99
step 1700, train accuracy 0.99
step 1800, train accuracy 0.98
step 1900, train accuracy 1


In [18]:
test_acc = accuracy.eval(feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})

In [19]:
test_acc

0.9832